In [ ]:
!pip install TeleBot
!pip install pyTelegramBotAPI
!pip install telegram
!pip install pydub

In [2]:
import os
import telebot
import telegram
import tensorflow as tf
import numpy as np
import pydub


In [29]:
bot = telebot.TeleBot("6370453544:AAEI0zlqAp9HaPkuQv3o448Qs5YoYN1qSbs")
model = tf.keras.models.load_model("audio_classification_model.h5")

In [ ]:
bot = telebot.TeleBot("6370453544:AAEI0zlqAp9HaPkuQv3o448Qs5YoYN1qSbs")
model = tf.keras.models.load_model("audio_classification_model.h5")

@bot.message_handler(commands=["start"])
def send_welcome(message):
    bot.send_message(message.chat.id , "Hello " + message.from_user.first_name + "\nWelcome to Audio Recognizer bot🎙🎵🔊" )
    help(message)

@bot.message_handler(commands=['help'])
def help(message):
    my_keyboard= telebot.types.ReplyKeyboardMarkup(row_width=3)
    key1 = telebot.types.KeyboardButton("/Friends_voice_recognition")
    key2 = telebot.types.KeyboardButton("/Singers_voice_recognition")
    my_keyboard.row(key1)
    my_keyboard.row(key2)
    bot.send_message(message.chat.id ,"Choose one of the below keys :" , reply_markup=my_keyboard)


@bot.message_handler(commands=['Friends_voice_recognition'])
def mymessage(message):
  message1 = bot.send_message(message.chat.id , "Now send a voice ... ")
  bot.register_next_step_handler(message1 , friends_voice_recognizer)

@bot.message_handler(content_types=['voice'])
def friends_voice_recognizer(message):
  file_id = message.voice.file_id
  new_file = bot.get_file(file_id)
  file_path = bot.get_file(file_id).file_path
  downloaded_file = bot.download_file(file_path)
  try:
      with open("/content/data/voice.ogg", 'wb') as new_file:
          new_file.write(downloaded_file)
  except Exception as e:
      bot.reply_to(message, "Error saving file.")
      return

  audio = pydub.AudioSegment.from_file("/content/data/voice.ogg")
  audio_array = audio.get_array_of_samples()
  silent_chunks = pydub.silence.split_on_silence(audio , min_silence_len=2000 , silence_thresh=-45)
  result = sum(silent_chunks)
  result.export("/content/wav_data/voice.wav")
  audio = pydub.AudioSegment.from_file("/content/wav_data/voice.wav")
  chunks = pydub.utils.make_chunks(audio , 1000)

  for i , chunk in enumerate(chunks) :
      if len(chunk) >= 1000 :
          chunk.export(os.path.join("/content/dataset/z" , "fvoice.wav") , format="wav")


  test_data = tf.keras.utils.audio_dataset_from_directory(
    "/content/dataset",
    shuffle=False,batch_size=2,output_sequence_length= 48000,ragged= False,
    labels="inferred",label_mode="categorical",sampling_rate= None,seed=59
  )

  classes_labels = np.array(['abdollah', 'azra', 'davood', 'javad', 'khadijeh', 'kiana','maryam', 'matin','melika', 'mohammad',
                             'mohammad_parvari','mona', 'nahid', 'nima', 'omid', 'parisa','parsa', 'sadegh', 'sajedeh', 'shima', 'tara'])
  predictions = model.predict(test_data)
  probability = predictions
  predicted_class_index = np.argmax(probability)
  print(predicted_class_index)
  predicted_class = classes_labels[predicted_class_index]
  print(predicted_class)
  print(f'The owner of this voice is : {predicted_class}')
  bot.send_message(chat_id= message.chat.id , text=f'The owner of this voice is  :  {predicted_class}')


#~~~~~~~~~~~~~~~~ singers audio recognition ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
@bot.message_handler(commands=['Singers_voice_recognition'])
def mymessage2(message):
  message2 = bot.send_message(message.chat.id , "Now send a song...")
  bot.register_next_step_handler(message2 , singers_voice_recognition)

@bot.message_handler(content_types=['audio'])
def singers_voice_recognition(message):
  bot.send_dice(chat_id=message.chat.id , emoji="🎲")



bot.infinity_polling()